In [1]:
import numpy as np
import json

In [2]:
from jamesbot.utils.embeddings import EmbeddingHandler
from jamesbot.slot_parser import SlotParser

In [3]:
embeddings = np.array(json.load(open('../jamesbot/data/processed/user_simulator/glove_embeddings.json', 'r')))
glove_dict = json.load(open('../jamesbot/data/processed/user_simulator/glove_dict.json', 'r'))
labels_dict = json.load(open('../jamesbot/data/processed/user_simulator/labels_dict.json', 'r'))
slots_dict = json.load(open('../jamesbot/data/processed/user_simulator/slots_dict.json', 'r'))
actions_dict = json.load(open('../jamesbot/data/processed/user_simulator/actions_dict.json', 'r'))
samples_embedded = json.load(open('../jamesbot/data/processed/user_simulator/samples_embedded.json', 'r'))
samples_data = json.load(open('../jamesbot/data/processed/user_simulator/samples.json', 'r'))
dialog_templates = json.load(open('../jamesbot/data/processed/user_simulator/dialog_templates.json', 'r'))
message_templates = json.load(open('../jamesbot/data/processed/user_simulator/message_templates.json', 'r'))

In [4]:
token_embeddings = EmbeddingHandler(glove_dict)
label_embeddings = EmbeddingHandler(labels_dict)
slot_embeddings = EmbeddingHandler(slots_dict)
action_embeddings = EmbeddingHandler(actions_dict)

In [5]:
slot_parser = SlotParser(
    token_embeddings, label_embeddings, action_embeddings, slot_embeddings,
    checkpoint_path = './models/gcloud_turn_parser_13/model.ckpt-14'
)

SlotParserModel:
Embeddings: [4320, 300]
IOB labels: 51
Present slots: 50
Actions: 16
INFO:tensorflow:Restoring parameters from ./models/gcloud_turn_parser_13/model.ckpt-14


In [9]:
slot_parser.parse('I am looking for a trip to toronto from frankfurt on 23 august.')

({'dst_city': 'toronto', 'or_city': 'frankfurt', 'str_date': '23 august'},
 ['inform'],
 ['dst_city', 'intent', 'or_city', 'str_date'])

In [97]:
class UserSimulator:
    
    INITIAL_SLOT_OPTIONS = {
        'or_city': 1.0,
        'dst_city': 0.8,
        'budget': 0.5,
        'n_adults': 0.5,
        'n_children': 0.2
    }
    
    REQUIRED_INFORM_SLOTS = [
        'or_city', 'dst_city', 'str_date', 'end_date', 'budget'
    ]
    
    REQUIRED_REQUEST_SLOTS = [
        'price', ''
    ]
    
    def __init__(self, slot_parser, dialog_templates, message_templates):
        self._slot_parser = slot_parser
        self._dialog_templates = dialog_templates
        self._message_templates = message_templates
        
        self._state = {}
        
    def _has_template(self, actions, slots):
        return self._template_key(actions, slots) in self._message_templates
        
    def _template_key(self, actions, slots):
        return '&'.join(sorted(actions))+'.'+'&'.join(sorted(slots))
    
    def _get_template(self, key):
        available_templates = self._message_templates[key]
        template_idx = np.random.randint(len(available_templates))
        return available_templates[template_idx]
        
    def reset(self):
        dialog_id = np.random.randint(len(self._dialog_templates))
        self._state = {
            'dialog': self._dialog_templates[dialog_id],
            'actions': [],
            'value_pointers': {},
        }
        print(self._state['dialog'])
        
    def _slot_value_idx(self, slot):
        if slot not in self._state['value_pointers']:
            self._state['value_pointers'][slot] = 0
        return self._state['value_pointers'][slot]
        
    def _has_slot(self, slot):
        return slot in self._state['dialog']['constraints']
    
    def _has_alternative_slot_value(self, slot):
        value_idx = self._slot_value_idx(slot)
        return len(self._state['dialog']['constraints'][slot]) > (value_idx + 1)
        
    def _slot_value(self, slot):
        return self._state['dialog']['constraints'][slot][self._slot_value_idx(slot)]
        
    def _slot_values(self, slots):
        return {slot: self._slot_value(slot) for slot in slots}
    
    def _replace_placeholders(self, template, values):
        for (slot, value) in values.items():
            template = template.replace('<SLOT.'+slot+'>', value)
        return template
        
    def _initial_inform_slots(self):
        initial_slots = set()
        constraints = self._state['dialog']['constraints']
        for (slot, slot_p) in self.INITIAL_SLOT_OPTIONS.items():
            if slot in constraints and constraints[slot][0] != '-1' and np.random.uniform() <= slot_p:
                initial_slots.add(slot)
        return initial_slots
            
    def initial_message(self):
        slots = self._initial_inform_slots()
        template_key = self._template_key(['inform'], slots)
        self._state['actions'].append((['inform'], slots))
        
        return self._replace_placeholders(
            self._get_template(template_key),
            self._slot_values(slots)
        )
    
    def _reward(self, slot_values, actions, mentioned_slots):
        last_action = self._state['actions'][-1][0]
        return 1
    
    def _request_response(self, slot_values, actions, mentioned_slots):
        if len(mentioned_slots) == 0:
            raise ValueError('No requested slots.')
        
        inform_slots = set()
        for slot in mentioned_slots:
            if self._has_slot(slot) and self._slot_value(slot) != '-1':
                inform_slots.add(slot)
        if len(inform_slots) == 0:
            return ['negate'], []
        else:
            return ['inform'], list(inform_slots)
        
    def _offer_response(self, slot_values, actions, mentioned_slots):
        pass
        
    
    def _no_result_response(self, slot_values, actions, mentioned_slots):
        return ['request_alts'], []
    
    def _suggest_response(self, slot_values, actions, mentioned_slots):
        return ['request_alts'], []
    
    def _next_action(self, slot_values, actions, mentioned_slots):
        print(slot_values, actions, mentioned_slots)
        
        if 'request' in actions:
            return self._request_response(slot_values, actions, mentioned_slots)
        
        if 'offer' in actions:
            return self._offer_response(slot_values, actions, mentioned_slots)
        
        if 'suggest' in actions:
            return self._no_result_response(slot_values, actions, mentioned_slots)
            
        if 'no_result' in actions:
            return self._suggest_response(slot_values, actions, mentioned_slots)
    
    def next_message(self, agent_response):
        parser_result = self._slot_parser.parse(agent_response)
        actions, slots = self._next_action(*parser_result)
        template_key = self._template_key(actions, slots)
        self._state['actions'].append((actions, slots))
        
        return (
            self._reward(*parser_result),
            self._replace_placeholders(
                self._get_template(template_key),
                self._slot_values(slots)
            )
        )
        
sim = UserSimulator(slot_parser, dialog_templates, message_templates)
sim.reset()
print(sim.initial_message())

while True:
    print(sim.next_message(input()))

{'constraints': {'dst_city': ['Brasilia'], 'str_date': ['September 30'], 'end_date': ['October 4'], 'or_city': ['Portland'], 'budget': ['2000']}, 'requests': []}
Portland
Where would you like to go from?
{} ['request'] ['dst_city', 'or_city']
(1, 'What about Portland to Brasilia?')
Do you have any dates in mind?
{} ['request'] ['end_date', 'str_date']
(1, 'Between September 30 and October 4.')
I can give you a 5 day stay in havana hotel for 1200 usd. Does that work?
{'duration': '5 day', 'dst_city': 'havana', 'price': '1200 usd'} ['offer'] ['<NO_SLOT>', 'dst_city', 'duration', 'price']


TypeError: 'NoneType' object is not iterable

In [96]:
message_templates['request.seat']

['That sounds lavish, what type of flight would that be?',
 'what type of flight is that going to be?',
 'and is that flight in <SLOT.seat> class?',
 'That sounds very nice. Does the package include a <SLOT.seat> class flight?',
 'what type of flight is that?',
 'and what are the classes of the flights?',
 'Will we be flying <SLOT.seat> class?',
 'Do you know from your database whether it is an economy flight or a business class flight?',
 'And are the flights economy or business class?',
 'and is the price for <SLOT.seat> flights']